# 作るもの
ドラえもんのキャラを判別する画像認識器を作成する。

今回はドラえもん、のび太、スネ夫、ジャイアンの４名を判別する。


# 初期設定
Colabで実行しているので、何度も`pip install`させないように、Google Driveにmoduleを落として、importする。

In [ ]:
# hide
# !pip install --target "/content/gdrive/MyDrive/Colab Notebooks/modules" -Uqq fastbook

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import sys, os
nb_path = '/content/modules'
os.symlink('/content/gdrive/MyDrive/Colab Notebooks/modules', nb_path)
sys.path.insert(0, nb_path)

In [ ]:
#hide
# !pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *
from fastai.vision.widgets import *

# 画像のダウンロード
BingSearchを利用して、画像をダウンロードする。

In [ ]:
# azureのキー
key = os.environ.get('AZURE_SEARCH_KEY', '')

In [ ]:
def my_search_images_bing(key, search_term):

    search_url = "https://api.bing.microsoft.com/v7.0/images/search"
    headers = {"Ocp-Apim-Subscription-Key" : key}
    min_size = 128

    params  = {"q": search_term,
               "imageType": "photo",
               "minHeight": min_size,
               "minWidth": min_size,
               "count": 150}
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()

    return L(search_results['value'])

search_images_bing = my_search_images_bing
search_images_bing

In [ ]:
image_types = ['doraemon','nobita', 'suneo', 'jaian']
path = Path('/content/gdrive/My Drive/Colab Notebooks/doraemon/images')

カテゴリー名でディレクトリを切って、各画像は、ディレクトリ毎にダウンロードさせる。

In [ ]:
if not path.exists():
    path.mkdir()
for o in image_types:
    dest = (path/o)
    dest.mkdir(exist_ok=True)
    results = search_images_bing(key, f'{o}')
    download_images(dest, urls=results.attrgot('contentUrl'))

In [ ]:
fns = get_image_files(path)
fns

破損している画像は、処理の対象から外す。

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink);

# DataLoaderの生成


In [ ]:
characters = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

# 学習

In [ ]:
characters = characters.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = characters.dataloaders(path)

In [ ]:
dls.train.n

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(10)

# 学習結果の確認

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(4, nrows=3)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
#hide
import os
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): 
    filepath = str(cleaner.fns[idx])
    ext = os.path.splitext(filepath)[-1]
    dest = path/cat/os.path.basename(filepath)
    i = 1
    while True:
        if os.path.exists(dest):
            filename = os.path.splitext(os.path.basename(dest))[0] + "_" + str(i)
            dest = path/cat/(filename+ext)
            i += 1
        else:
            break
    shutil.move(str(cleaner.fns[idx]), dest)

# 学習結果のエクスポート
モデルのアーキテクチャと学習済みパラメータを、エクスポートする。

In [ ]:
path = Path('/content/gdrive/My Drive/Colab Notebooks/doraemon')
learn.export(path/'export.pkl')

# 予測


In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
im = Image.open(path/'doraemon.png')
im.to_thumb(128,128)

In [ ]:
learn_inf.predict(path/'doraemon.png')